# Traffic Sign Classification with Tensorflow


## Step 0: Load Image Dataset

In [7]:
import pickle
import os
from tqdm import tqdm
from zipfile import ZipFile

# Define variables
setnames = []
data     = dict()
X        = dict()
y        = dict()

path = "/home/tiwi/Documents/mldata/"
file = "traffic-signs-data.zip"


print("Filenames in data container: \n")
for filename in ZipFile(path+file).namelist():
    print(filename)
    # Create setnames from filenames: e.g. "train", "test", ...
    setnames.append(os.path.split(filename)[1].rpartition('.')[0])
    # Get full path + file for each pickle file
    with open(path + file.rpartition('.')[0] + '/' + filename, mode='rb') as f:
        data[setnames[-1]] = pickle.load(f)
        
for setname in setnames:
    X[setname], y[setname] = data[setname]['features'], data[setname]['labels']
        

Filenames in data container: 

test.p
train.p
valid.p
